In [99]:
import jieba
import logging
import os
import sys
import multiprocessing

from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences

import warnings
import gensim

In [100]:
def seg_words():
    origin_file = "data/cleaned_tex.tex"  # 原始文件
    stop_words_file = "data/hit_stopwords.txt"  # 停用词路径
    user_dict_file = "data/jieba_dict.txt"  # 用户自定义词典路径
    stop_words = list()
    # 加载停用词
    with open(stop_words_file, 'r', encoding="utf8") as stop_words_file_object:
        contents = stop_words_file_object.readlines()
        for line in contents:
            line = line.strip()
            stop_words.append(line)
    # 加载用户字典(停用)
    jieba.load_userdict(user_dict_file)
    target_file = open("data/cut_word.txt", 'w', encoding="utf-8")
    with open(origin_file, 'r', encoding="utf-8") as origin_file_object:
        contents = origin_file_object.readlines()
        for line in contents:
            line = line.strip()
            out_str = ''
            word_list = jieba.cut(line, cut_all=False)
            for word in word_list:
                if word not in stop_words:
                    if word != "\t":
                        out_str += word
                        out_str += ' '
            target_file.write(out_str.rstrip() + "\n")
    target_file.close()
    print("end")
seg_words()

end


In [101]:
def train_model():
    program = os.path.basename(sys.argv[0])
    logger = logging.getLogger(program)
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info("running %s" % ' '.join(sys.argv))

    input_dir = 'data/cut_word.txt'
    outp1 = 'model/vector.model'
    outp2 = 'model/vector.vector'

    model = Word2Vec(PathLineSentences(input_dir),
                     size=300, window=10, min_count=2,
                     workers=multiprocessing.cpu_count(), iter=200,sg=1)
    model.save(outp1)	# 存储二进制模型文件
    model.wv.save_word2vec_format(outp2, binary=False)	# 存储类似于数组的模型文件
train_model()

2023-06-03 11:14:48,711: INFO: running /Users/lizixuan/opt/miniconda3/envs/WorkForMyFirstPaper/lib/python3.8/site-packages/ipykernel_launcher.py --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"573ae252-ea93-4e87-bf64-8f58621a7a65" --shell=9002 --transport="tcp" --iopub=9004 --f=/Users/lizixuan/Library/Jupyter/runtime/kernel-v2-35093s1etL1gsRGkI.json
2023-06-03 11:14:48,712: INFO: files read into PathLineSentences:data/cut_word.txt
2023-06-03 11:14:48,712: INFO: collecting all words and their counts
2023-06-03 11:14:48,713: INFO: reading file data/cut_word.txt
2023-06-03 11:14:48,713: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-06-03 11:14:48,715: INFO: collected 1617 word types from a corpus of 12099 raw words and 495 sentences
2023-06-03 11:14:48,715: INFO: Loading a fresh vocabulary
2023-06-03 11:14:48,716: INFO: effective_min_count=2 retains 899 unique words (55% of original 1617, drops 71

In [102]:
warnings.filterwarnings(action='ignore', category=UserWarning,module='gensim')
model = gensim.models.Word2Vec.load("model/vector.model")

word = '集合'
result = model.wv.most_similar(word)

print("跟 "+word+" 最相近的词：")
for i in result:
    print(i)

2023-06-03 11:14:58,163: INFO: loading Word2Vec object from model/vector.model
2023-06-03 11:14:58,175: INFO: loading wv recursively from model/vector.model.wv.* with mmap=None
2023-06-03 11:14:58,176: INFO: setting ignored attribute vectors_norm to None
2023-06-03 11:14:58,176: INFO: loading vocabulary recursively from model/vector.model.vocabulary.* with mmap=None
2023-06-03 11:14:58,176: INFO: loading trainables recursively from model/vector.model.trainables.* with mmap=None
2023-06-03 11:14:58,177: INFO: setting ignored attribute cum_table to None
2023-06-03 11:14:58,177: INFO: loaded model/vector.model
2023-06-03 11:14:58,178: INFO: precomputing L2-norms of word weight vectors


跟 集合 最相近的词：
('笛卡儿积', 0.4557154178619385)
('空集', 0.4438934326171875)
('幂集', 0.4135299324989319)
('关系图', 0.413086473941803)
('关系矩阵', 0.4034850299358368)
('文氏图', 0.39920979738235474)
('对应', 0.3672679662704468)
('排斥原理', 0.36523038148880005)
('判别', 0.363705039024353)
('构成', 0.36069250106811523)


In [103]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_product = norm(vector1) * norm(vector2)
    similarity = dot_product / norm_product
    return similarity

vec1 = model['运算'] - model['二元运算']
vec2 = model['度数'] - model['出度']

cosine_similarity(vec1,vec2)

/var/folders/1l/pvfnlvjd7pj3qlyv3pkmn5nr0000gn/T/ipykernel_46448/2172002816.py:10: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).

/var/folders/1l/pvfnlvjd7pj3qlyv3pkmn5nr0000gn/T/ipykernel_46448/2172002816.py:11: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



0.02612512

In [104]:
result = model.wv.similar_by_vector(model['出度'] -  model['度数'] + model['无向图'],topn=20)
result

/var/folders/1l/pvfnlvjd7pj3qlyv3pkmn5nr0000gn/T/ipykernel_46448/1566801876.py:1: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



[('出度', 0.6843885779380798),
 ('人度', 0.6823106408119202),
 ('无向图', 0.6396973133087158),
 ('欧拉通路', 0.5436170101165771),
 ('内点', 0.49236705899238586),
 ('连通', 0.48218441009521484),
 ('无', 0.4809902012348175),
 ('无向边', 0.47857657074928284),
 ('欧拉回路', 0.4665376842021942),
 ('有向图', 0.46574366092681885),
 ('恰好', 0.4649372100830078),
 ('小', 0.45772847533226013),
 ('可达', 0.4547189772129059),
 ('达', 0.45470327138900757),
 ('重合', 0.44618773460388184),
 ('奇度顶点', 0.44201231002807617),
 ('终点', 0.44170570373535156),
 ('每条', 0.44157668948173523),
 ('单向连通图', 0.4403216242790222),
 ('层数', 0.4267460107803345)]

In [105]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)
plt.show()
